In [1]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))

In [2]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=13)

<IB connected to 127.0.0.1:7496 clientId=13>

In [3]:
import stock

tsla = stock.StockX(ib, "TSLA")

tsla.get_fundamentals()
tsla.fundamentals.__dict__

{'sector': 'Unclassified',
 'industry': 'Unclassified',
 'sub_industry': 'Unclassified',
 'latest_available_date': '2024-09-30',
 'price_currency': 'USD',
 'reporting_currency': 'USD',
 'exchange_rate': 1.0,
 'current_price': 369.49,
 'high_52week': 375.43,
 'low_52week': 138.8025,
 'pricing_date': '2024-12-05T00:00:00',
 'volume_10day_avg': 66.88714,
 'enterprise_value': 1160133.0,
 'market_cap': 1186085.0,
 'revenue_ttm': 97150.0,
 'ebitda': 13338.0,
 'net_income_ttm': 12743.0,
 'eps_ttm': 3.64717,
 'revenue_per_share': 27.83867,
 'book_value_per_share': 21.80574,
 'cash_per_share': 10.49205,
 'cash_flow_per_share': 5.12014,
 'dividend_per_share': 0.0,
 'gross_margin': 18.22851,
 'operating_margin': 0.0,
 'net_profit_margin': 0.0,
 'revenue_growth_rate': 0.0,
 'eps_growth_rate': 0.0,
 'pe_ratio': 101.3087,
 'price_to_book': 16.94462,
 'price_to_sales': 0.0,
 'employee_count': 140473,
 'forecast': ForecastMetrics(consensus_recommendation=2.7391, target_price=209.8933, projected_growth

In [4]:
tsla.get_daily_data()

Loading data from data/historical_data_store/TSLA_1_day.csv
Stored data: 250 rows of data
Loading data from data/historical_data_store/TSLA_1_day.csv


,open,high,low,close,volume
date,,,,,
2023-12-06,240.61,246.57,238.85,240.37,1080540
2023-12-07,240.80,244.08,236.98,242.91,916952
2023-12-08,242.00,245.27,239.27,243.23,878583
2023-12-11,242.00,245.00,237.45,239.55,819699
2023-12-12,240.25,241.22,233.87,236.48,785221
...,...,...,...,...,...
2024-11-26,338.00,346.96,335.21,340.17,470824
2024-11-27,339.76,342.60,326.59,334.40,441879
2024-11-29,336.00,348.50,334.65,345.44,272292


In [5]:
tsla.set_default_daily_ta()
tsla.frame.data

,open,high,low,close,volume,MA_cl_200,MA_cl_50,MA_vo_10,ACC_close,VDEV_10,TDUR_MA_cl_50
date,,,,,,,,,,,
2023-12-06,240.61,246.57,238.85,240.37,1080540,NaN,NaN,NaN,NaN,NaN,1
2023-12-07,240.80,244.08,236.98,242.91,916952,NaN,NaN,NaN,NaN,NaN,1
2023-12-08,242.00,245.27,239.27,243.23,878583,NaN,NaN,NaN,NaN,NaN,1
2023-12-11,242.00,245.00,237.45,239.55,819699,NaN,NaN,NaN,NaN,NaN,1
2023-12-12,240.25,241.22,233.87,236.48,785221,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
2024-11-26,338.00,346.96,335.21,340.17,470824,213.43710,269.0762,723829.0,65.76,-34.953698,45
2024-11-27,339.76,342.60,326.59,334.40,441879,214.18910,271.1760,671255.0,48.25,-34.171217,46
2024-11-29,336.00,348.50,334.65,345.44,272292,214.96490,273.2024,605475.7,21.40,-55.028418,47


In [6]:

tsla.compute_daily_filters_with_scores()


,BRK_UP_MA,BRK_UP_MA,ABV_close_MA_cl_50,ABV_close_MA_cl_200,ABV_3_TDUR_MA_cl_50,ABV_80_VDEV_10,filter_score,all_true
date,,,,,,,,
2023-12-06,False,False,False,False,True,False,1,False
2023-12-07,False,False,False,False,True,False,1,False
2023-12-08,False,False,False,False,True,False,1,False
2023-12-11,False,False,False,False,True,False,1,False
2023-12-12,False,False,False,False,True,False,1,False
...,...,...,...,...,...,...,...,...
2024-11-26,False,False,True,True,False,True,3,False
2024-11-27,False,False,True,True,False,True,3,False
2024-11-29,False,False,True,True,False,True,3,False


In [7]:
ib.disconnect()

In [8]:
from data import historical_data as hd
from ib_insync import *


tdata = hd.get_hist_data('TSLA', "52 weeksAgo", 'now', '1 day')
tdata

Loading data from data/historical_data_store/TSLA_1_day.csv
Stored data: 250 rows of data
Loading data from data/historical_data_store/TSLA_1_day.csv


,open,high,low,close,volume
date,,,,,
2023-12-06,240.61,246.57,238.85,240.37,1080540
2023-12-07,240.80,244.08,236.98,242.91,916952
2023-12-08,242.00,245.27,239.27,243.23,878583
2023-12-11,242.00,245.00,237.45,239.55,819699
2023-12-12,240.25,241.22,233.87,236.48,785221
...,...,...,...,...,...
2024-11-26,338.00,346.96,335.21,340.17,470824
2024-11-27,339.76,342.60,326.59,334.40,441879
2024-11-29,336.00,348.50,334.65,345.44,272292


In [ ]:
mansfield, roc, combined = analyze_sector(
    etf_symbol='XLE',
    lookback_period="26 weeksAgo",
    mansfield_period=150,
    roc_period=150,
    ma_short=20,
    ma_long=100,
    verbose=True
)

In [17]:

    



# Example Usage
# Assuming `etf_data` and `market_data` are DataFrames with the required columns


from data import historical_data as hd
import compare



etf_data = hd.get_hist_data('XLF', "52 weeksAgo", 'now', '1 day')
market_data = hd.get_hist_data('SPY', "52 weeksAgo", 'now', '1 day')

analysis = compare.SectorAnalysis(etf_data, market_data)
analysis.compute_all(
    mansfield_period=200,  # Period for Mansfield RSI
    roc_period=200,         # Period for MA ROC ratio
    ma_short=50,           # Short MA period
    ma_long=200           # Long MA period
)
result_df = analysis.get_df()
today_metrics = analysis.get_today(verbose=True)
display(today_metrics)
result_df


Loading data from data/historical_data_store/XLF_1_day.csv
Stored data: 249 rows of data
Loading data from data/historical_data_store/XLF_1_day.csv
Loading data from data/historical_data_store/SPY_1_day.csv
Stored data: 250 rows of data
Loading data from data/historical_data_store/SPY_1_day.csv

=== Today's Market Analysis ===

Mansfield RSI: 4.25
MA ROC Ratio: 1.16
Combined Score: 2.71

📈 BULLISH


mansfield_rsi     4.254055
ma_roc_ratio      1.160618
combined_score    2.707337
dtype: float64

,etf_50_ma,etf_200_ma,market_50_ma,market_200_ma,mansfield_rsi,ma_roc_ratio,combined_score
date,,,,,,,
2024-09-25,43.9020,41.07935,551.0202,520.40360,-0.554947,0.000000,-0.277474
2024-09-26,43.9246,41.12345,551.3066,520.96265,-0.521474,-0.003399,-0.262437
2024-09-27,43.9616,41.16730,551.6480,521.50450,0.199075,0.125981,0.162528
2024-09-30,44.0124,41.21035,552.1318,522.04710,0.014426,0.025382,0.019904
2024-10-01,44.0514,41.24860,552.4014,522.52575,0.510626,0.061587,0.286106
2024-10-02,44.0928,41.28670,552.7696,523.01615,0.406175,-0.079116,0.163530
2024-10-03,44.1360,41.32415,553.2764,523.50465,0.084966,-0.144174,-0.029604
2024-10-04,44.1912,41.36600,553.9360,524.00725,1.017609,0.274168,0.645889
2024-10-07,44.2238,41.40340,554.4046,524.47240,0.609759,0.092630,0.351194


In [5]:
etf_data

,open,high,low,close,volume
date,,,,,
2023-12-08,35.97,36.19,35.83,36.14,311722
2023-12-11,36.13,36.42,36.09,36.38,287274
2023-12-12,36.35,36.66,36.29,36.66,298369
2023-12-13,36.69,37.41,36.55,37.41,525548
2023-12-14,37.28,37.71,37.23,37.55,622257
...,...,...,...,...,...
2024-11-27,51.26,51.62,51.18,51.33,289386
2024-11-29,51.51,51.57,51.21,51.26,138172
2024-12-02,51.35,51.49,50.70,50.86,477889


In [3]:
from ib_insync import IB, Stock
import xml.etree.ElementTree as ET

contract = Stock('TSLA', 'SMART', 'USD')

# fundamental_data = ib.reqFundamentalData(contract, 'ReportsFundamentals')
fundamental_data = ib.reqFundamentalData(contract, 'ReportSnapshot')
root = ET.fromstring(fundamental_data)

In [5]:
def print_xml_tree(element, indent=0):
    """Print the XML tree structure with indentation"""
    # Print current element
    print('  ' * indent + f'<{element.tag}>', end='')
    
    # Print attributes if any
    if element.attrib:
        print(f' {element.attrib}', end='')
        
    # Print text content if it exists and isn't whitespace
    if element.text and element.text.strip():
        print(f' = {element.text.strip()}')
    else:
        print()
        
    # Print all children recursively
    for child in element:
        print_xml_tree(child, indent + 1)

print_xml_tree(root)

<ReportSnapshot> {'Major': '1', 'Minor': '0', 'Revision': '1'}
  <CoIDs>
    <CoID> {'Type': 'RepNo'} = C8279
    <CoID> {'Type': 'CompanyName'} = Tesla Inc
    <CoID> {'Type': 'IRSNo'} = 912197729
    <CoID> {'Type': 'CIKNo'} = 0001318605
    <CoID> {'Type': 'OrganizationPermID'} = 5088024644
  <Issues>
    <Issue> {'ID': '1', 'Type': 'C', 'Desc': 'Common Stock', 'Order': '1'}
      <IssueID> {'Type': 'Name'} = Ordinary Shares
      <IssueID> {'Type': 'Ticker'} = TSLA
      <IssueID> {'Type': 'RIC'} = TSLA.O
      <IssueID> {'Type': 'DisplayRIC'} = TSLA.OQ
      <IssueID> {'Type': 'InstrumentPI'} = 67910050
      <IssueID> {'Type': 'QuotePI'} = 72106022
      <Exchange> {'Code': 'NASD', 'Country': 'USA'} = NASDAQ
      <GlobalListingType> = OSR
      <MostRecentSplit> {'Date': '2022-08-25'} = 3.00003
    <Issue> {'ID': '2', 'Type': 'P', 'Desc': 'Preferred Stock', 'Order': '1'}
      <IssueID> {'Type': 'Name'} = Preference Shares Series B
      <IssueID> {'Type': 'InstrumentPI'} = 7447

In [14]:
from ib_insync import IB, Stock, Index, Contract
import xml.etree.ElementTree as ET

contract = Stock('SXLK', 'SMART', 'USD')
contract.secType = 'STK'

fundamental_data = ib.reqFundamentalData(contract, 'ReportSnapshot')
root2 = ET.fromstring(fundamental_data)

print_xml_tree(root2)

TypeError: a bytes-like object is required, not 'list'

In [9]:
import requests
from bs4 import BeautifulSoup

# URL of the page
url = "https://www.ssga.com/uk/en_gb/investor/prospectuses-etf-information?"

# Custom headers to mimic a real browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

# Send a GET request
response = requests.get(url, headers=headers)

# Check the response
if response.status_code == 200:
    print("Successfully fetched the HTML!")
    html_content = response.text
    
    # Parse with BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    print(soup.prettify()[:1000])  # Print the first 1000 characters to inspect the HTML
else:
    print(f"Failed to fetch the HTML. Status code: {response.status_code}")


Successfully fetched the HTML!
<!DOCTYPE HTML>
<html lang="en-GB">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <title>
   Prospectuses and ETF Information
  </title>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <!-- None-critical site css -->
  <link href="/etc.clientlibs/ssmp/clientlibs/clientlib-site/ssmp-global.lc-1731709572502202411-lc.min.css" rel="stylesheet" type="text/css"/>
  <!-- OneTrust Cookies Consent Notice start for ssga.com -->
  <script>
   // akam-sw.js install script version 1.3.6
"serviceWorker"in navigator&&"find"in[]&&function(){var e=new Promise(function(e){"complete"===document.readyState||!1?e():(window.addEventListener("load",function(){e()}),setTimeout(function(){"complete"!==document.readyState&&e()},1e4))}),n=window.akamServiceWorkerInvoked,r="1.3.6";if(n)aka3pmLog("akam-setup already invoked");else{window.akamServiceWorkerInvoked=!0,window.aka3pmLog=function(){window.akamServiceWorkerDebug&&

In [11]:
import requests
import pandas as pd

# Direct URL to the Excel file

url_xlk = "https://www.ssga.com/uk/en_gb/investor/library-content/products/fund-data/etfs/emea/holdings-daily-emea-en-zpdt-gy.xlsx" # technology
url_xlf = "https://www.ssga.com/uk/en_gb/investor/library-content/products/fund-data/etfs/emea/holdings-daily-emea-en-zpdf-gy.xlsx" # financials
url_xlv = "https://www.ssga.com/uk/en_gb/investor/library-content/products/fund-data/etfs/emea/holdings-daily-emea-en-zpdh-gy.xlsx" # health care
url_xle = "https://www.ssga.com/uk/en_gb/investor/library-content/products/fund-data/etfs/emea/holdings-daily-emea-en-zpde-gy.xlsx" # energy
url_xly = "https://www.ssga.com/uk/en_gb/investor/library-content/products/fund-data/etfs/emea/holdings-daily-emea-en-zpdd-gy.xlsx" # consumer discretionary
url_xlc = "https://www.ssga.com/uk/en_gb/investor/library-content/products/fund-data/etfs/emea/holdings-daily-emea-en-zpdk-gy.xlsx" # communication services
url_xlu = "https://www.ssga.com/uk/en_gb/investor/library-content/products/fund-data/etfs/emea/holdings-daily-emea-en-zpdu-gy.xlsx" # utilities
url_xlre = "https://www.ssga.com/uk/en_gb/investor/library-content/products/fund-data/etfs/emea/holdings-daily-emea-en-zpdk-gy.xlsx" # real estate


https://www.ssga.com/uk/en_gb/investor/library-content/products/fund-data/etfs/emea/holdings-daily-emea-en-zpdk-gy.xlsx

# Custom headers to mimic a real browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

# Send GET request
response = requests.get(url, headers=headers)

if response.status_code == 200:
    # Save the Excel file
    with open("etf_holdings.xlsx", "wb") as f:
        f.write(response.content)
    print("Successfully downloaded the Excel file!")
    
    # Optionally, read the Excel file into a pandas DataFrame
    df = pd.read_excel("etf_holdings.xlsx")
    print("\nFirst few rows of the data:")
    print(df.head())
    
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

Successfully downloaded the Excel file!


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_trbc_table():
    # URL of the Wikipedia page
    url = "https://en.wikipedia.org/wiki/The_Refinitiv_Business_Classification"
    
    # Fetch the page content
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the classification table
    table = soup.find('table', {'class': 'wikitable'})
    
    # Initialize lists to store the data
    hierarchical_ids = []
    economic_sectors = []
    business_sectors = []
    industry_groups = []
    industries = []
    activities = []
    
    # Extract data from table rows
    for row in table.find_all('tr')[1:]:  # Skip header row
        cols = row.find_all('td')
        if cols:  # Skip empty rows
            # Extract text from each column, strip whitespace
            hierarchical_id = cols[-1].get_text(strip=True) if len(cols) >= 6 else ''
            economic_sector = cols[0].get_text(strip=True) if len(cols) >= 1 else ''
            business_sector = cols[1].get_text(strip=True) if len(cols) >= 2 else ''
            industry_group = cols[2].get_text(strip=True) if len(cols) >= 3 else ''
            industry = cols[3].get_text(strip=True) if len(cols) >= 4 else ''
            activity = cols[4].get_text(strip=True) if len(cols) >= 5 else ''
            
            # Append to lists
            hierarchical_ids.append(hierarchical_id)
            economic_sectors.append(economic_sector)
            business_sectors.append(business_sector)
            industry_groups.append(industry_group)
            industries.append(industry)
            activities.append(activity)
    
    # Create DataFrame
    df = pd.DataFrame({
        'Hierarchical_ID': hierarchical_ids,
        'Economic_Sector': economic_sectors,
        'Business_Sector': business_sectors,
        'Industry_Group': industry_groups,
        'Industry': industries,
        'Activity': activities
    })
    
    # Forward fill the text columns
    columns_to_fill = ['Economic_Sector', 'Business_Sector', 'Industry_Group', 'Industry']
    df[columns_to_fill] = df[columns_to_fill].fillna(method='ffill')
    
    return df

# Execute the scraping and processing
try:
    df = scrape_trbc_table()
    print("Table scraped successfully!")
    print("\nFirst few rows of the processed table:")
    print(df.head())
    
    # Optionally save to CSV
    df.to_csv('trbc_classification.csv', index=False)
    print("\nTable saved to 'trbc_classification.csv'")
    
except Exception as e:
    print(f"An error occurred: {str(e)}")

Table scraped successfully!

First few rows of the processed table:
  Hierarchical_ID Economic_Sector        Business_Sector Industry_Group  \
0      5010101010          Energy  Energy - Fossil Fuels           Coal   
1      5010101011                                                         
2      5010101012                                                         
3      5010201010                                             Oil & Gas   
4      5010202010                                                         

                               Industry  \
0                                  Coal   
1                                         
2                                         
3                  Integrated Oil & Gas   
4  Oil & Gas Exploration and Production   

                                     Activity  
0                                  Coal (NEC)  
1                         Coal Mining Support  
2                              Coal Wholesale  
3                        Integ

In [10]:
import pandas as pd

def process_classification_data(input_file='trbc_classification.csv', output_file='trbc_classification_filled.csv'):
    """
    Read the TRBC classification CSV, forward fill empty cells, and save the processed data.
    """
    # Read the CSV file
    df = pd.read_csv(input_file)
    
    # Forward fill the columns in order
    columns_to_fill = [
        'Economic Sector',
        'Business Sector',
        'Industry Group',
        'Industry'
    ]
    
    # Forward fill each column
    for column in columns_to_fill:
        df[column] = df[column].fillna(method='ffill')
    
    # Remove any rows that might still have NaN values in critical columns
    df = df.dropna(subset=['Activity', 'Hierarchical ID'])
    
    # Reset the index after dropping rows
    df = df.reset_index(drop=True)
    
    # Save the processed data
    df.to_csv(output_file, index=False)
    
    # Print some validation information
    print(f"Original shape: {pd.read_csv(input_file).shape}")
    print(f"Processed shape: {df.shape}")
    print(f"\nSample of processed data:")
    print(df.head())
    print(f"\nSaved processed data to {output_file}")
    
    return df

# Example usage
df = process_classification_data()

# Optional: Display value counts for each column to verify the fill worked correctly
def verify_data(df):
    print("\nValue counts for each column:")
    for column in df.columns:
        print(f"\n{column}:")
        print(f"Unique values: {df[column].nunique()}")
        print(f"Null values: {df[column].isnull().sum()}")

verify_data(df)

Original shape: (895, 6)
Processed shape: (895, 6)

Sample of processed data:
  Economic Sector        Business Sector Industry Group  \
0          Energy  Energy - Fossil Fuels           Coal   
1          Energy  Energy - Fossil Fuels           Coal   
2          Energy  Energy - Fossil Fuels           Coal   
3          Energy  Energy - Fossil Fuels      Oil & Gas   
4          Energy  Energy - Fossil Fuels      Oil & Gas   

                               Industry  \
0                                  Coal   
1                                  Coal   
2                                  Coal   
3                  Integrated Oil & Gas   
4  Oil & Gas Exploration and Production   

                                     Activity  Hierarchical ID  
0                                  Coal (NEC)       5010101010  
1                         Coal Mining Support       5010101011  
2                              Coal Wholesale       5010101012  
3                        Integrated Oil & Gas   

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def scrape_sic_codes():
    # URL of the Companies House SIC codes page
    url = "https://resources.companieshouse.gov.uk/sic/"
    
    try:
        # Make the request
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes
        
        # Create BeautifulSoup object
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Initialize lists to store data
        sections = []
        codes = []
        descriptions = []
        section_codes = []
        section_names = []
        
        # Find the table containing SIC codes
        table = soup.find('table')
        
        if table:
            current_section = None
            current_section_code = None
            
            # Iterate through table rows
            for row in table.find_all('tr'):
                # Get all cells in the row
                cells = row.find_all(['td', 'th'])
                
                if len(cells) >= 2:
                    # Check if this is a section header
                    first_cell_text = cells[0].get_text().strip()
                    if first_cell_text.startswith('Section'):
                        current_section = cells[1].get_text().strip()
                        current_section_code = first_cell_text.split()[-1]
                    # Check if this is a SIC code row
                    elif re.match(r'^\d{4,5}$', first_cell_text):
                        codes.append(first_cell_text)
                        descriptions.append(cells[1].get_text().strip())
                        section_codes.append(current_section_code)
                        section_names.append(current_section)
        
        # Create DataFrame
        df = pd.DataFrame({
            'Section Code': section_codes,
            'Section Name': section_names,
            'SIC Code': codes,
            'Description': descriptions
        })
        
        # Save to CSV
        df.to_csv('uk_sic_codes.csv', index=False)
        print(f"Successfully scraped {len(df)} SIC codes")
        return df
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the webpage: {e}")
        return None
    except Exception as e:
        print(f"Error processing the data: {e}")
        return None

def validate_sic_codes(df):
    """
    Validate the scraped data for completeness and consistency
    """
    # Check for missing values
    missing_values = df.isnull().sum()
    
    # Check for unique sections
    unique_sections = df['Section Code'].unique()
    
    # Check for code format consistency
    invalid_codes = df[~df['SIC Code'].str.match(r'^\d{4,5}$')]
    
    print("\nValidation Results:")
    print(f"Total number of records: {len(df)}")
    print(f"Number of unique sections: {len(unique_sections)}")
    print("\nMissing values:")
    print(missing_values)
    
    if len(invalid_codes) > 0:
        print("\nInvalid SIC codes found:")
        print(invalid_codes)
    else:
        print("\nAll SIC codes are in valid format")

if __name__ == "__main__":
    print("Starting SIC code scraper...")
    df = scrape_sic_codes()
    
    if df is not None:
        validate_sic_codes(df)
        print("\nData has been saved to 'uk_sic_codes.csv'")

Starting SIC code scraper...
Successfully scraped 731 SIC codes

Validation Results:
Total number of records: 731
Number of unique sections: 21

Missing values:
Section Code    0
Section Name    0
SIC Code        0
Description     0
dtype: int64

All SIC codes are in valid format

Data has been saved to 'uk_sic_codes.csv'


In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def get_sector_code(naics_code):
    # Return the first two digits of the NAICS code as the sector code
    return naics_code[:2] if naics_code else None

def get_industry_group_code(naics_code):
    # Return the first four digits of the NAICS code as the industry group code
    return naics_code[:4] if naics_code else None

def clean_naics_code(code):
    # Remove any non-numeric characters and whitespace
    if code:
        return re.sub(r'\D', '', code.strip())
    return None

import time

def scrape_naics_table():
    url = "https://www.naics.com/six-digit-naics/"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'DNT': '1'
    }
    
    try:
        # Add a small delay before the request
        time.sleep(2)
        
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        # Verify we got HTML content
        if 'text/html' not in response.headers.get('Content-Type', ''):
            raise ValueError("Did not receive HTML content from the server")
            
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table')
        
        # Check if table exists
        if not table:
            raise ValueError("Could not find table in the webpage")
        
        # Initialize lists to store the data
        data = []
        
        # Iterate through all rows in the table
        rows = table.find_all('tr')[1:]  # Skip header row
        
        for row in rows:
            cols = row.find_all('td')
            if cols:
                naics_code = clean_naics_code(cols[0].text)
                industry_name = cols[1].text.strip()
                
                if naics_code:
                    sector_code = get_sector_code(naics_code)
                    industry_group_code = get_industry_group_code(naics_code)
                    
                    data.append({
                        'Sector Code': sector_code,
                        'Sector Name': 'TBD',  # Would need a mapping for sector names
                        'Industry Group Code': industry_group_code,
                        'Industry Group Name': 'TBD',  # Would need a mapping for group names
                        'Industry Code': naics_code,
                        'Industry Name': industry_name,
                        'Sub-Industry Code': naics_code if len(naics_code) == 6 else None,
                        'Sub-Industry Name': industry_name if len(naics_code) == 6 else None
                    })
        
        # Create DataFrame
        df = pd.DataFrame(data)
        
        # Export to CSV
        df.to_csv('naics_data.csv', index=False)
        print("Data has been successfully scraped and saved to 'naics_data.csv'")
        
    except requests.RequestException as e:
        print(f"Error fetching the webpage: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")


scrape_naics_table()

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


An error occurred: Could not find table in the webpage


In [18]:
import requests
from bs4 import BeautifulSoup

url = "https://www.naics.com/six-digit-naics/"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

# First, let's check the network tab in browser dev tools to see what request is made when expanding
# Let's try printing the HTML around the table to see its structure
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Let's look for elements with specific IDs or classes that might contain our table
print("Looking for relevant elements:")
print("\nElements with ID 'naics-table' or similar:")
naics_table = soup.find(id=lambda x: x and 'naics' in x.lower())
if naics_table:
    print(naics_table.prettify()[:500])

print("\nElements with class containing 'table':")
table_elements = soup.find_all(class_=lambda x: x and 'table' in x.lower())
for element in table_elements:
    print(element.prettify()[:1000])

Looking for relevant elements:

Elements with ID 'naics-table' or similar:
<link href="https://www.naics.com/wp-content/plugins/naics-api/includes/naicsApiStyle.css?ver=9.6" id="naicsAPIStyle-css" media="all" rel="stylesheet"/>


Elements with class containing 'table':
<div class="content-area grid-parent mobile-grid-100 grid-100 tablet-grid-100" id="primary">
 <main class="site-main" id="main">
  <article class="post-1608 page type-page status-publish" id="post-1608" itemscope="" itemtype="https://schema.org/CreativeWork">
   <div class="inside-article">
    <div class="entry-content" itemprop="text">
     <input id="qid" name="qid" type="hidden" value="070ed92328"/>
     <div class="fl-builder-content fl-builder-content-1608 fl-builder-content-primary fl-builder-global-templates-locked" data-post-id="1608">
      <div class="fl-row fl-row-full-width fl-row-bg-photo fl-node-kzuigbo9f8p6 fl-row-default-height fl-row-align-center fl-row-bg-overlay" data-node="kzuigbo9f8p6">
       <div 